# NVIDIA FLARE Message Serialization

One important design consideration in distribution system is message serialization. Python Pickle is the default object serialization mechanism used in many Python and machine learning frameworks. However, Python pickle is considered "unsecure" for message serialization in distributed systems by security teams.

NVFLARE uses a secure mechanism called FOBS (Flare OBject Serializer) for message serialization and deserialization when exchanging data between the server and clients.

# Flare Object Serializer (FOBS)

## Overview
FOBS is a drop-in replacement for Pickle for security purposes. It uses [MessagePack](https://msgpack.org/index.html) to serialize objects.

FOBS sacrifices convenience for security. With Pickle, most objects are supported automatically using introspection. To serialize an object using FOBS, a Decomposer must be registered for the class. Several decomposers for commonly used classes are pre-registered with the module.

FOBS supports enum types by registering decomposers automatically for all classes that are subclasses of Enum.

FOBS treats all other classes as dataclass by registering a generic decomposer for dataclasses. Dataclass is a class whose constructor only changes the state of the object without side-effects. Side-effects include changing global variables, creating network connection, files etc.

FOBS throws TypeError exception when it encounters an object with no decomposer registered. For example,

TypeError: can not serialize 'xxx' object

## Usage

FOBS defines following 4 functions, similar to Pickle,

```dumps(obj)```: Serializes obj and returns bytes

```dump(obj, stream)```: Serializes obj and writes the result to stream

```loads(data)```: Deserializes the data and returns an object

```load(stream)```: Reads data from stream and deserializes it into an object

Examples,

```
from nvflare.fuel.utils import fobs
data = fobs.dumps(dxo)
new_dxo = fobs.loads(data)
```


**Pickle/json compatible functions:**

```

data = fobs.dumps(shareable)
new_shareable = fobs.loads(data)
```


## Decomposers

Decomposers are classes that inherit abstract base class fobs.Decomposer. FOBS uses decomposers to break an object into serializable objects before serializing it using MessagePack.

Decomposers are very similar to serializers, except that they don’t have to convert object into bytes directly, they can just break the object into other objects that are serializable.

An object is serializable if its type is supported by MessagePack or a decomposer is registered for its class.

FOBS recursively decomposes objects till all objects are of types supported by MessagePack. Decomposing looping must be avoided, which causes stack overflow. Decomposers form a loop when one class is decomposed into another class which is eventually decomposed into the original class. For example, this scenario forms the simplest loop: X decomposes into Y and Y decomposes back into X.
 
All classes defined in fobs/decomposers folder are automatically registered. Other decomposers must be registered manually like this,

```
fobs.register(FooDecomposer)
fobs.register(BarDecomposer())
```

fobs.register takes either a class or an instance as the argument. Decomposer whose constructor takes arguments must be registered as instance.

A decomposer can either serialize the class into bytes or decompose it into objects of serializable types. In most cases, it only involves saving members as a list and reconstructing the object from the list.

MessagePack can’t handle items larger than 4GB in dict. To work around this issue, FOBS can externalize the large item and just stores a reference in the buffer. DatumManager is used to handle the externalized data. For most objects which don’t deal with dict items larger than 4GB, the DatumManager is not needed.

Here is an example of a simple decomposer. Even though datetime is not supported by MessagePack, a decomposer is included in fobs module so no need to further decompose it.

from nvflare.fuel.utils import fobs
```

class Simple:

    def __init__(self, num: int, name: str, timestamp: datetime):
        self.num = num
        self.name = name
        self.timestamp = timestamp


class SimpleDecomposer(fobs.Decomposer):

    def supported_type(self) -> Type[Any]:
        return Simple

    def decompose(self, obj, manager) -> Any:
        return [obj.num, obj.name, obj.timestamp]

    def recompose(self, data: Any, manager) -> Simple:
        return Simple(data[0], data[1], data[2])


fobs.register(SimpleDecomposer)
data = fobs.dumps(Simple(1, 'foo', datetime.now()))
obj = fobs.loads(data)
assert obj.num == 1
assert obj.name == 'foo'
assert isinstance(obj.timestamp, datetime)
```
The same decomposer can be registered multiple times. Only first one takes effect, the others are ignored with a warning message.

Note that fobs_initialize() may need to be called if decomposers are not registered.

 